In [8]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy.special
import glob

pd.set_option('display.max_columns',85)
pd.set_option('display.max_rows',100)

In [9]:
# read in data to panda

InPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/mOTU/"
file="motus_Merge_genus_032024"
OutPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/"

df=pd.read_csv(InPath+file+".txt",header=2,delimiter="\t")

threshold = 0.01
#df

# Threshold: 
The threshold set here decides which genus/species to ignore! If at least in 1 sample the threshold is reached, we keep the species!

In [10]:
# kick out rows that only contain 0:
## do this by first adding a column that includes the sum of samples that have this genus

rowNum = df.shape[0]

sumSamples = []
for i in range(rowNum):
    a  = np.array(df.iloc[i])[1:].astype(float)
    aa = np.count_nonzero(a > threshold)
    sumSamples.append(aa)

sumSamples = np.array(sumSamples)

## and add it as column to the data frame:
#df.insert(0, "sumSamples", sumSamples)
#df

df_collect = df
df_collect

,consensus_taxonomy,Gent_bb_15_1_woCE_IR.genus.motus,Gent_bb_15_2_woCE_IR.genus.motus,Gent_bb_15_3_woCE_IR.genus.motus,Gent_bb_15_4_woCE_IR.genus.motus,Gent_bb_15_5_woCE_IR.genus.motus,Gent_bb_1_1_woCE_IR.genus.motus,Gent_bb_1_2_woCE_IR.genus.motus,Gent_bb_1_3_woCE_IR.genus.motus,Gent_bb_1_4_woCE_IR.genus.motus,Gent_iso_15_1_woCE_IR.genus.motus,Gent_iso_15_2_woCE_IR.genus.motus,Gent_iso_15_3_woCE_IR.genus.motus,Gent_iso_15_4_woCE_IR.genus.motus,NIOO_A00_15_1_woCE_IR.genus.motus,NIOO_A00_15_2_woCE_IR.genus.motus,NIOO_A00_15_3_woCE_IR.genus.motus,NIOO_A00_15_4_woCE_IR.genus.motus,NIOO_A00_15_5_woCE_IR.genus.motus,NIOO_A00_1_1_woCE_IR.genus.motus,NIOO_A00_1_2_woCE_IR.genus.motus,NIOO_A00_1_3_woCE_IR.genus.motus,NIOO_A00_1_4_woCE_IR.genus.motus,NIOO_A00_1_5_woCE_IR.genus.motus,NIOO_M00_15_1_woCE_IR.genus.motus,NIOO_M00_15_3_woCE_IR.genus.motus,NIOO_M00_15_4_woCE_IR.genus.motus,NIOO_M00_15_5_woCE_IR.genus.motus,NIOO_M00_1_1_woCE_IR.genus.motus,NIOO_M00_1_3_woCE_IR.genus.motus,NIOO_M00_1_4_woCE_IR.genus.motus,NIOO_M00_1_5_woCE_IR.genus.motus,NIOO_bb_15_1_woCE_IR.genus.motus,NIOO_bb_15_3_woCE_IR.genus.motus,NIOO_bb_15_4_woCE_IR.genus.motus,NIOO_bb_1_1_woCE_IR.genus.motus,NIOO_bb_1_3_woCE_IR.genus.motus,NIOO_bb_1_4_woCE_IR.genus.motus,RUG_bb_15_1_woCE_IR.genus.motus,RUG_bb_15_2_woCE_IR.genus.motus,RUG_bb_15_3_woCE_IR.genus.motus,RUG_bb_15_4_woCE_IR.genus.motus,...,VU_bb_0_2_woCE_IR.genus.motus,VU_bb_0_3_woCE_IR.genus.motus,VU_bb_0_4_woCE_IR.genus.motus,VU_bb_0_5_woCE_IR.genus.motus,VU_bb_15_1_woCE_IR.genus.motus,VU_bb_15_2_woCE_IR.genus.motus,VU_bb_15_3_woCE_IR.genus.motus,VU_bb_15_4_woCE_IR.genus.motus,VU_bb_15_5_woCE_IR.genus.motus,VU_bb_1_1_woCE_IR.genus.motus,VU_bb_1_2_woCE_IR.genus.motus,VU_bb_1_3_woCE_IR.genus.motus,VU_bb_1_4_woCE_IR.genus.motus,VU_bb_1_5_woCE_IR.genus.motus,VU_bn50_0_1_woCE_IR.genus.motus,VU_bn50_0_2_woCE_IR.genus.motus,VU_bn50_0_3_woCE_IR.genus.motus,VU_bn50_0_4_woCE_IR.genus.motus,VU_bn50_0_5_woCE_IR.genus.motus,VU_bn50_15_1_woCE_IR.genus.motus,VU_bn50_15_2_woCE_IR.genus.motus,VU_bn50_15_3_woCE_IR.genus.motus,VU_bn50_15_4_woCE_IR.genus.motus,VU_bn50_15_5_woCE_IR.genus.motus,VU_bn5_0_1_woCE_IR.genus.motus,VU_bn5_0_2_woCE_IR.genus.motus,VU_bn5_0_3_woCE_IR.genus.motus,VU_bn5_0_4_woCE_IR.genus.motus,VU_bn5_0_5_woCE_IR.genus.motus,VU_bn5_15_1_woCE_IR.genus.motus,VU_bn5_15_2_woCE_IR.genus.motus,VU_bn5_15_3_woCE_IR.genus.motus,VU_bn5_15_4_woCE_IR.genus.motus,VU_bn5_15_5_woCE_IR.genus.motus,WUR_bb_15_1_woCE_IR.genus.motus,WUR_bb_15_2_woCE_IR.genus.motus,WUR_bb_15_3_woCE_IR.genus.motus,WUR_bb_1_1_woCE_IR.genus.motus,WUR_bb_1_2_woCE_IR.genus.motus,WUR_bb_1_3_woCE_IR.genus.motus,WUR_bleach_15_1_woCE_IR.genus.motus,WUR_bleach_15_2_woCE_IR.genus.motus
0,Cellvibrio,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,Papillibacter,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000

In [11]:
# now only keep the rows that have a non zero entry

dropIdx = np.where(sumSamples == 0)[0]

df_nonZero = df.drop(dropIdx)
df_nonZero

df_nonZero.set_index('consensus_taxonomy',inplace=True)

df_2 = df_nonZero.transpose()

df_2.shape

(93, 22)

In [6]:
# also, get the sample names as colums

institution = []
treatment = []
week = []
rep = []
newRowName = []

rowNames = df_2.index.tolist()

for i in rowNames:
    nameList = i.split('_')
    institution.append(nameList[0])
    treatment.append(nameList[1])
    week.append(nameList[2])
    rep.append(nameList[3])

    newRowName.append(nameList[0] + "_" + nameList[1] + "_" + nameList[2] + "_" + nameList[3])

#add columns to the dataframe
df_final=df_2
df_final.insert(0, "institution", institution)
df_final.insert(1, "treatment", treatment)
df_final.insert(2, "week", week)
df_final.insert(2, "replicate", rep)
df_final.insert(0, "name_short", newRowName)


In [7]:
# Sort and tidy up the dataframe
df_final=df_final.sort_values(['institution','treatment','week','replicate'])
df_final

consensus_taxonomy,name_short,institution,treatment,replicate,week,Roseomonas,Bacillus,Massilia,Rhodococcus,Streptomyces,Plantibacter,Achromobacter,Paracoccus,Pseudomonas,Microbacterium,Pseudochrobactrum,Stenotrophomonas,Brevundimonas,Paenibacillus,Ochrobactrum,Escherichia,Kocuria,Chryseobacterium,Serratia,Pseudoxanthomonas,Moraxellaceae gen. [Enhydrobacter/Moraxella],unassigned
Gent_bb_1_1_woCE_IR.genus.motus,Gent_bb_1_1,Gent,bb,1,1,0.000000,0.024455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.824973,0.000000,0.002584,0.000000,0.000000,0.000000,0.134876,0.000000,0.000000,0.000000,0.000000,0.000000,0.013111
Gent_bb_1_2_woCE_IR.genus.motus,Gent_bb_1_2,Gent,bb,2,1,0.000000,0.008731,0.000000,0.000253,0.000000,0.000000,0.000000,0.000000,0.003570,0.968262,0.000000,0.004296,0.000000,0.000000,0.000654,0.005642,0.000000,0.000000,0.001425,0.000000,0.000000,0.001589
Gent_bb_1_3_woCE_IR.genus.motus,Gent_bb_1_3,Gent,bb,3,1,0.000000,0.013997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000640,0.968560,0.000000,0.000346,0.000000,0.000000,0.000000,0.013558,0.000000,0.000000,0.000000,0.000000,0.000000,0.000336
Gent_bb_1_4_woCE_IR.genus.motus,Gent_bb_1_4,Gent,bb,4,1,0.000000,0.014073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.957582,0.000000,0.003645,0.000000,0.000000,0.000000,0.004835,0.000000,0.000000,0.000000,0.000000,0.000000,0.014471
Gent_bb_15_1_woCE_IR.genus.motus,Gent_bb_15_1,Gent,bb,1,15,0.000000,0.000000,0.003042,0.000000,0.003585,0.000000,0.000000,0.000000,0.788890,0.194397,0.000000,0.000000,0.000000,0.000000,0.000000,0.007364,0.000000,0.000000,0.000000,0.000000,0.000000,0.002723
Gent_bb_15_2_woCE_IR.genus.motus,Gent_bb_15_2,Gent,bb,2,15,0.000000,0.001874,0.003961,0.000000,0.230041,0.000000,0.000000,0.002025,0.000000,0.583732,0.000000,0.000000,0.000000,0.000000,0.000000,0.161882,0.000000,0.000000,0.000000,0.000000,0.000000,0.016486
Gent_bb_15_3_woCE_IR.genus.motus,Gent_bb_15_3,Gent,bb,3,15,0.000000,0.000000,0.000000,0.000000,0.309457,0.000000,0.000000,0.000000,0.000000,0.623868,0.000000,0.001684,0.000000,0.000000,0.000000,0.058603,0.000000,0.000000,0.000000,0.000000,0.000000,0.005238
Gent_bb_15_4_woCE_IR.genus.motus,Gent_bb_15_4,Gent,bb,4,15,0.000000,0.000000,0.006273,0.000000,0.345753,0.000000,0.000000,0.000000,0.000000,0.580123,0.000000,0.002608,0.000000,0.000000,0.000000,0.058733,0.000000,0.000000,0.000000,0.000000,0.000000,0.006511
Gent_bb_15_5_woCE_IR.genus.motus,Gent_bb_15_5,Gent,bb,5,15,0.000000,0.001119,0.000000,0.000000,0.361133,0.000000,0.000000,0.430002,0.000000,0.157090,0.000000,0.000000,0.000000,0.000000,0.000000,0.029193,0.000000,0.000000,0.000000,0.000000,0.000000,0.017750
Gent_iso_15_1_woCE_IR.genus.motus,Gent_iso_15_1,Gent,iso,1,15,0.000000,0.002635,0.000713,0.000000,0.000000,0.000000,0.000000,0.018837,0.000847,0.963522,0.000000,0.006171,0.001069,0.000000,0.000185,0.003524,0.000000,0.000000,0.000258,0.000000,0.000000,0.000933


In [8]:
df_final.to_csv(OutPath+file+"_0.01minFrac_FromPY.csv", index=False) #
#df2=pd.read_csv("data.csv")

# To Do:
- is there a better wa than the transpose one?
- get rid of the 'consensus_taxonomy' entry